<a href="https://colab.research.google.com/github/ayoubbensakhria/finance_algo/blob/master/MA_and_Momentum_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Moving Average

Moving average is a series of averages points between many fixed subsets of a dataset. 

## Simple Moving Average
The unweighted mean of the previous k data-points.

\begin{aligned}
S M A_{k} &=\frac{p_{n-k+1}+p_{n-k+2} \cdots+p_{n}}{k} \\
&=\frac{1}{k} \sum_{i=n-k+1}^{n} p_{i}
\end{aligned}

## Cumulative Moving Average
The data arrive in an ordered datum stream, and the user would like to get the average of all of the data up until the current datum
\begin{aligned}
C M A_{n+1} &=\frac{x_{n+1}+n \cdot C M A_{n}}{n+1} \\
&=\frac{x_{n+1}+(n+1-1) \cdot C M A_{n}}{n+1} \\
&=\frac{(n+1) \cdot C M A_{n}+x_{n+1}-C M A_{n}}{n+1} \\
&=C M A_{n}+\frac{x_{n+1}-C M A_{n}}{n+1}
\end{aligned}

Where :
\begin{equation}
C M A_{n}=\frac{x_{1}+\cdots+x_{n}}{n}
\end{equation}

## Weighted Moving Average

The average that has multiplying factors to give different weights to data at different positions in the sample window. Mathematically, the weighted moving average is the convolution of the data with a fixed weighting function.

\begin{equation}
\mathrm{WMA}_{M}=\frac{n p_{M}+(n-1) p_{M-1}+\cdots+2 p_{((M-n)+2)}+p_{((M-n)+1)}}{n+(n-1)+\cdots+2+1}
\end{equation}

## Exponential Moving Average

The first-order infinite impulse response filter that applies weighting factors which decrease exponentially. The weighting for each older datum decreases exponentially, never reaching zero. The graph at right shows an example of the weight decrease.

$$
\mathrm{EMA}_{\text {today }}=\frac{p_{1}+(1-\alpha) p_{2}+(1-\alpha)^{2} p_{3}+(1-\alpha)^{3} p_{4}+\cdots}{1+(1-\alpha)+(1-\alpha)^{2}+(1-\alpha)^{3}+\cdots}
$$
since $1 / \alpha=1+(1-\alpha)+(1-\alpha)^{2}+\cdots$.

# Momentum
Momentum investing is a system of buying stocks or other securities that have had high returns over the past three to twelve months, and selling those that have had poor returns over the same period.